In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!ls

adc.json  drive  sample_data


In [0]:
cd drive/Houston

/content/drive/Houston/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
area_hsi = []
for i in range(3):
    for j in range(14):
        img_path = "area/pc_area_" + str(i) + "_" + str(j) + ".png"
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        area_hsi.append(img)
        
area_hsi = np.array(area_hsi)

In [0]:
print(area_hsi.shape)

(42, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
area_hsi_train_data = []
area_hsi_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            area_hsi_train_data.append(area_hsi[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            area_hsi_test_data.append(area_hsi[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
area_hsi_train_data = np.array(area_hsi_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [0]:
print(area_hsi_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 42)
(2832,)
(12197,)


In [0]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [0]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
HSI_PATCH_SIZE = 27
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [0]:
padded_area_hsi = np.lib.pad(area_hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
print(padded_area_hsi.shape)

(42, 375, 1931)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
area_hsi_train_patches = []
area_hsi_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      area_hsi_train_patches.append(get_patches(padded_area_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      area_hsi_test_patches.append(get_patches(padded_area_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))

In [0]:
area_hsi_train_patches = np.array(area_hsi_train_patches)
area_hsi_test_patches = np.array(area_hsi_test_patches)
print(area_hsi_train_patches.shape)
print(area_hsi_test_patches.shape)

(2832, 27, 27, 42)
(12197, 27, 27, 42)


In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = area_hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

area_hsi_model = Sequential()
area_hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
area_hsi_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
area_hsi_model.add(BatchNormalization())
area_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
area_hsi_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
area_hsi_model.add(BatchNormalization())
area_hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


area_hsi_model.add(Flatten())
area_hsi_model.add(Dense(FC1, activation='relu'))
area_hsi_model.add(Dropout(0.6))
area_hsi_model.add(Dense(FC2, activation='relu'))
area_hsi_model.add(Dropout(0.4))
area_hsi_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
area_hsi_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

area_hsi_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 14, 14, 500)       756500    
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 500)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 500)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 4, 100)         1250100   
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 100)         400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
__________

In [0]:
history = area_hsi_model.fit(area_hsi_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0218 - acc: 0.9940
Epoch 2/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0319 - acc: 0.9901
Epoch 3/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0255 - acc: 0.9922
Epoch 4/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0110 - acc: 0.9975
Epoch 5/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0215 - acc: 0.9933
Epoch 6/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0344 - acc: 0.9915
Epoch 7/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0122 - acc: 0.9954
Epoch 8/50
2832/2832 [==============================] - 4s 1ms/step - loss: 0.0183 - acc: 0.9940
Epoch 9/50
2832/2832 [==============================] - 3s 1ms/step - loss: 0.0312 - acc: 0.9919
Epoch 10/50
2832/2832 [==============================] - 5s 2ms/step - loss: 0.0238 - acc: 0.9922
Epoch 11/50
2832/2832 [======

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [0]:
test_cls = test_labels - 1
prediction = area_hsi_model.predict(area_hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.5921128146265475
Average Accuracy of training samples :  0.6549613043382394
Kappa statistics of training samples :  0.5571260555629782
Each class accuracy of training samples :  [0.67088608 0.54845361 0.71828909 0.58993711 0.9106204  0.46440678
 0.73265306 0.38493038 0.41775837 0.98270893 0.5140799  0.59696262
 0.54029851 0.96482412 0.78761062]
Confusion matrix : [[583  91   0 165   0   0  21   0   1   0   0   0   0   0   8]
 [192 532   0 162  31   6  44   3   0   0   0   0   0   0   0]
 [  0   0 487   0   0   0   0   0   0   0   0   0   0   0 191]
 [ 23 134   0 469  49   0 117   0   0   0   0   0   0   0   3]
 [  1   0   0  14 866   0   0   7  22   0   0  12   0   3  26]
 [ 82   0   0   6   0 137   1  10   3   0   0   0   0   0  56]
 [  0  60   0 123   1   0 718  37  34   0   0   2   0   5   0]
 [ 20  31   0  14  23   0  55 470  76  23  28 334  46  13  88]
 [151 216   0  87  86   0  64 142 861 208 131  63  19  29   4]
 [  0   0   0   0   0   0

In [0]:
area_hsi_model.save_weights('Models/area_hsi_model_weights.h5')
